**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
!pip install scikit-video

In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import random
import tqdm as tqdm

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Dropout
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, Conv3D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization, Flatten

Using TensorFlow backend.


In [3]:
import warnings
warnings.filterwarnings("ignore") # only if needed

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [4]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [5]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

Given a state `s`, the `act` function will return the action to take. But here we will not return our best action everytime (learned from the past), instead thanks to `epsilon` (the exploration parameter) we will take sometime (with probability `epsilon`) a random action. This parameter `epsilon` is essential in order to explore other options and also to try to get out of a local minima.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [6]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4 # just extend so we can code the borders (limits of the env.)
        self.grid_size = grid_size # save the grid size
        self.max_time = max_time # T, how much time before the game is over
        self.temperature = temperature # probability of having a cheese, poison "in a given cell"

        #board on which one plays
        self.board = np.zeros((grid_size, grid_size)) # stores the bonus/malus (and so the cheese/poison positions): reward
        self.position = np.zeros((grid_size, grid_size)) # the board (what the rat sees/ where is the rat/borders)

        # coordinate of the rat
        self.x = 0 # starting position x
        self.y = 1 # starting position y

        # self time
        self.t = 0 # initialize the time t=0

        self.scale = 16 # scale factor along the vertical axis (OpenCV)

        self.to_draw = np.zeros((max_time + 2, grid_size * self.scale, grid_size * self.scale, 3)) # for the video


    def draw(self, e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw) # display the video (e: look for the number of epoch)

    def get_frame(self, t):
        b = np.zeros((self.grid_size, self.grid_size, 3)) + 128 # neutral cells are grey (128, 128, 128) in rgb
        b[self.board > 0, 0] = 255 # cheese cells are yellow (255, 255, 0) in rgb
        b[self.board > 0, 1] = 255 # cheese cells are yellow (255, 255, 0) in rgb
        b[self.board > 0, 2] = 0 # cheese cells are yellow (255, 255, 0) in rgb
        b[self.board < 0, 0] = 255 # poison cells are red (255, 0, 0) in rgb
        b[self.board < 0, 1] = 0 # poison cells are red (255, 0, 0) in rgb
        b[self.board < 0, 2] = 0 # poison cells are red (255, 0, 0) in rgb
        b[self.x, self.y, 0] = 0 # rat's position is green (0, 128, 0) in rgb
        b[self.x, self.y, 2] = 0 # rat's position is green (0, 128, 0) in rgb
        b[self.x, self.y, 1] = 128 # rat's position is green (0, 128, 0) in rgb
        b[-2:, :, :] = 0 # limit of the board is black
        b[:, -2:, :] = 0 # limit of the board is black
        b[:2, :, :] = 0 # limit of the board is black
        b[:, :2, :] = 0 # limit of the board is black
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t, :, :, :] = b # save the frame


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t)) # stores the frame at time t

        self.position = np.zeros((self.grid_size, self.grid_size)) # the board (what the rat sees/ where is the rat/borders)

        
                                     ## -1 -1 -1 -1 -1 -1 -1 -1
                                     ## -1 -1 -1 -1 -1 -1 -1 -1
        self.position[0:2, :] = -1   ## -1 -1  ok  ok ok  -1 -1
        self.position[:, 0:2] = -1   ## -1 -1  ok  ok ok  -1 -1
        self.position[-2:, :] = -1   ## -1 -1  ok  ok ok  -1 -1
        self.position[:, -2:] = -1   ## -1 -1  ok  ok ok  -1 -1
                                     ## -1 -1 -1 -1 -1 -1 -1 -1
                                     ## -1 -1 -1 -1 -1 -1 -1 -1


        self.position[self.x, self.y] = 1 # 1 where is the rat
        if action == 0: # right
            if self.x == self.grid_size - 3: # if on the last column
                self.x = self.x-1 # we go to the left
            else:
                self.x = self.x + 1 # we go the right
        elif action == 1: # left
            if self.x == 2: # if on the 2th column
                self.x = self.x+1 # we go to the right
            else:
                self.x = self.x-1 # we go to the left
        elif action == 2: # down
            if self.y == self.grid_size - 3: # if on the last row
                self.y = self.y - 1 # we go up
            else:
                self.y = self.y + 1 # we go down
        elif action == 3: # up
            if self.y == 2: # if on the 2th row
                self.y = self.y + 1 # we go down
            else:
                self.y = self.y - 1 # we go up
        else:
            RuntimeError('Error: action not recognized') 

        self.t = self.t + 1 # update the time
        reward = self.board[self.x, self.y] # reward defines by the position
        self.board[self.x, self.y] = 0 # we reset the reward at our position (we ate the poison/cheese)
        game_over = self.t > self.max_time # game is over if t > T
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size, 1), # state of the game (board, position)
                                self.position.reshape(self.grid_size, self.grid_size, 1)), axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :] # we can only see the area around the rat (5 * 5)
        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        ### initialize a random position for the rat
        self.x = np.random.randint(3, self.grid_size - 3)
        self.y = np.random.randint(3, self.grid_size - 3)

        ### initialize the cheese
        bonus = 0.5 * np.random.binomial(1, self.temperature, size=self.grid_size ** 2)
        bonus = bonus.reshape(self.grid_size, self.grid_size)

        ### initialize the poison
        malus = - 1.0 * np.random.binomial(1, self.temperature, size=self.grid_size ** 2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        ### clipboard animation initialization
        self.to_draw = np.zeros((self.max_time + 2, self.grid_size * self.scale, self.grid_size * self.scale, 3))

        ### if a cheese overlap with a poison we let the cheese and cancel the poison
        malus[bonus > 0] = 0

        ### construct the final board using the bonus (cheese) and malus (poison)
        self.board = bonus + malus

        ### initialize the position
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2, :]= -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        ###  initialize the position of the rat (he eats the cheese or the poison!)
        self.board[self.x, self.y] = 0

        ### initialize the time
        self.t = 0

        ### construct the state using (board, position)
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),
                               axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :] # we only see 5 * 5 around the (self.x, self.y)
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [7]:
# parameters
size = 13 # grid size
T = 200 # how much time before the game is over
temperature = 0.3 # probability of having a cheese/poison at a specific location
epochs_train = 10 # set small when debugging
epochs_test = 10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

With `position` we store the board (limits of the environment) and we use it in order to move the rat. In the other hand `board` is used to store the reward (bonus/malus) and we use it when the rat eats.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [8]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.choice(4) # choose uniformly in {1, 2, 3, 4}

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [9]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
    
    for e in range(epochs):
        
        ##### FILL IN HERE
        state = env.reset() # first reset the environment
        game_over = False # initialize 'game_over'
        win, lose = 0, 0 # initialize win/lose
        while not game_over: # loop while it's not over
            if prefix == 'random':
                action = agent.learned_act(state) # take action with the agent
            elif prefix == 'cnn_test_explore':
                action = agent.learned_act(state.flatten().reshape(1, 5, 5, 3)) # take action with the agent
            elif prefix == 'fc_train':
                action = agent.learned_act(state.flatten().reshape(1, -1)) # take action with the agent
            else:
                action = agent.learned_act(state.reshape(1, 5, 5, 2)) # take action with the agent
            
            if prefix == 'cnn_test_explore':
                state, reward, game_over = env.act(action, train=False) # update the env. with the action
            else:
                state, reward, game_over = env.act(action) # update the env. with the action
            if reward > 0: # update win/lose
                win += reward
            else:
                lose -= reward

        # Save as a mp4
        env.draw(prefix + str(e))

        # Update stats
        score = score + win - lose

        print(f"Epoch {e}, win/lose count {win}/{lose}, average score ({np.round(score / (1 + e), 2)})")
    print(f'Final score: {score / epochs}')

In [10]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

# run the agent
test(agent, env, epochs_test, prefix='random')

# display the result
print("\nCheese in yellow and poison in red")
HTML(display_videos('random0.mp4')) # don't need to look other videos since it is random

Epoch 0, win/lose count 10.5/12.0, average score (-1.5)
Epoch 1, win/lose count 9.0/28.0, average score (-10.25)
Epoch 2, win/lose count 10.0/8.0, average score (-6.17)
Epoch 3, win/lose count 8.0/7.0, average score (-4.38)
Epoch 4, win/lose count 8.5/14.0, average score (-4.6)
Epoch 5, win/lose count 9.0/14.0, average score (-4.67)
Epoch 6, win/lose count 7.5/19.0, average score (-5.64)
Epoch 7, win/lose count 10.0/10.0, average score (-4.94)
Epoch 8, win/lose count 9.5/20.0, average score (-5.56)
Epoch 9, win/lose count 11.0/16.0, average score (-5.5)
Final score: -5.5

Cheese in yellow and poison in red


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




Starting from the definition:

$$ Q^\pi(s,a) = E_{p^{\pi}}\left[\sum_{t=0}^T\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a\right] = E_{p^{\pi}}\left[ r(s, a) + \sum_{t=1}^T \gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a\right] = E_{p^{\pi}}\left[ r(s, a) + \gamma\sum_{t=0}^T \gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a'\right] = E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]$$

As $Q^*(s,a)= \underset{\pi}{\max} Q^\pi(s,a)$ thus the only "problem" is inside the expectation. In fact, we will not compute the expectation given $(s', a') \sim p(.|s,a)$ now because as we look over all action to find the maximum $Q$ value. Therefore we just compute the expectation with $s'\sim p(.|s, a)$ and we have the result.

Finally a plausible objective is just to compute the mean squared error of the target $Q$, which is $r+\gamma\max\max_{a'}Q(s',a',\theta)$, and the prediction of our model, which is $Q(s,a,\theta)$ (because we just want to find a fixed point). N.b. here $\theta$ are all the parameters of the model we use to compute/estimate the $Q$ function/values.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [11]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory # maximum elements stored
        self.memory = list() # initialize the memory

    def remember(self, m):
        if len(self.memory) <= self.max_memory: # if not full
            self.memory.append(m) # store element m at the end
        else:
            self.memory = self.memory[1:] # remove the first element
            self.memory.append(m) # store element m at the end

    def random_access(self, batch_size):
        return random.sample(self.memory, batch_size) # random sample, from memory, of size batch_size

***
The pipeline we will use for training is given below:

In [12]:
def train(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in tqdm.tqdm_notebook(range(epoch)):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()

        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            if prefix == 'fc_train':
                action = agent.learned_act(state.flatten().reshape(1, -1)) # take action with the agent
            else:
                action = agent.learned_act(state.reshape(1, 5, 5, 2)) # take action with the agent

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over, prefix)
        

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix + str(e))

        # Update stats
        score += win - lose

        print(f"Epoch {e}/{epoch}, loss {round(np.float64(loss), 4)}, win/lose count {win}/{lose} ({win - lose})")
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [20]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon=0.1, memory_size=100, batch_size=16, n_state=2):
        super(DQN, self).__init__(epsilon=epsilon)
        self.epsilon = epsilon 

        # Discount for Q learning (gamma)
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        if np.random.rand() >= self.epsilon: # epsilon exploration strategy
            return np.argmax(agent.model.predict(s), axis=1) # our best action
        else:
            return np.random.choice(4) # random action

    def reinforce(self, state, next_state, action, reward, game_over, prefix):
        ### Two steps: first memorize the states, second learn from the pool
        ### 1) memorize
        self.memory.remember([state, next_state, action, reward, game_over])

        ### 2) learn from the pool
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))

        if len(self.memory.memory) < self.batch_size: # if not enaugh elements in memory we do nothing
            return 1e5 # unknow (loss)
        
        samples =  self.memory.random_access(self.batch_size) # random samples from the memory
        for i in range(self.batch_size):
            
            ### get elements from memory[i]
            input_states[i], next_s, a, r, end = samples[i] # state, next_state, action, reward, game_over
                
            ### update the target
            if end:
                target_q[i, a] = r 
            else:
                ### compute max_a Q(nex_state, a) using the model
                if prefix == 'fc_train':
                    Q_next_state = np.max(agent.model.predict(next_s.flatten().reshape(1, -1)))
                elif prefix == 'cnn_train_explore':
                    Q_next_state = np.max(agent.model.predict(next_s.flatten().reshape(1, 5, 5, 3)))
                else:
                    Q_next_state = np.max(agent.model.predict(next_s.reshape(1, 5, 5, 2)))
                    
                ### r + gamma * max_a Q(nex_state, a)
                target_q[i, a] = r + self.discount * Q_next_state

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        ### train the model on the batch
        if prefix == 'fc_train':
            input_data = np.array([input_states[i].flatten().reshape(-1) for i in range(self.batch_size)])
            loss = self.model.train_on_batch(input_data, target_q)
        elif prefix == 'cnn_train_explore':
            input_data = np.array([input_states[i].flatten().reshape(5, 5, 3) for i in range(self.batch_size)])
            loss = self.model.train_on_batch(input_data, target_q)
        else:
            input_data = np.array([input_states[i].reshape(5, 5, 2) for i in range(self.batch_size)]) 
            loss = self.model.train_on_batch(input_data, target_q)
            
        return loss

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1, **kwargs):
        super(DQN_FC, self).__init__( *args, **kwargs)
        
        ### NN Model
        model = Sequential() 

        #Input layer
        model.add(Dense(units=200, input_dim=50, activation='relu'))
        model.add(Dropout(0.25))

        #Hidden layer 1
        model.add(Dense(units=200, activation='relu'))
        model.add(Dropout(0.5))

        #Output layer
        model.add(Dense(units=4, activation=None))
        
        ####### FILL IN
        
        model.compile(adam(lr=lr), "mse")
        self.model = model

In [21]:
epochs_train = 21 # set small when debugging
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=0.001, epsilon=0.1, memory_size=2000, batch_size=32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train20.mp4'))

Epoch 0/21, loss 0.006, win/lose count 6.5/2.0 (4.5)
Epoch 1/21, loss 0.0006, win/lose count 2.0/2.0 (0.0)
Epoch 2/21, loss 0.001, win/lose count 1.0/8.0 (-7.0)
Epoch 3/21, loss 0.0019, win/lose count 3.5/2.0 (1.5)
Epoch 4/21, loss 0.0002, win/lose count 5.0/4.0 (1.0)
Epoch 5/21, loss 0.0024, win/lose count 5.0/3.0 (2.0)
Epoch 6/21, loss 0.0079, win/lose count 6.0/4.0 (2.0)
Epoch 7/21, loss 0.0053, win/lose count 5.5/3.0 (2.5)
Epoch 8/21, loss 0.004, win/lose count 3.5/2.0 (1.5)
Epoch 9/21, loss 0.0036, win/lose count 7.0/3.0 (4.0)
Epoch 10/21, loss 0.0032, win/lose count 6.5/8.0 (-1.5)
Epoch 11/21, loss 0.0056, win/lose count 5.5/5.0 (0.5)
Epoch 12/21, loss 0.0065, win/lose count 6.5/5.0 (1.5)
Epoch 13/21, loss 0.0103, win/lose count 7.5/5.0 (2.5)
Epoch 14/21, loss 0.0099, win/lose count 17.5/5.0 (12.5)
Epoch 15/21, loss 0.0015, win/lose count 14.5/6.0 (8.5)
Epoch 16/21, loss 0.0053, win/lose count 16.5/6.0 (10.5)
Epoch 17/21, loss 0.0104, win/lose count 12.5/5.0 (7.5)
Epoch 18/21, lo

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [22]:
class DQN_CNN(DQN):
    def __init__(self, *args, lr=0.1, n_state=2, **kwargs):
        super(DQN_CNN, self).__init__(*args, **kwargs)
        
        self.n_state = n_state
        ###### FILL IN
        ### create model
        model = Sequential()
        model.add(Conv2D(16, kernel_size=2, activation='relu', input_shape=(5, 5, n_state)))
#         model.add(Conv2D(32, kernel_size=2, activation='relu'))
        model.add(Flatten())
        model.add(Dense(32, activation='relu'))
        model.add(Dense(4, activation=None))
        
        model.compile(adam(lr=lr), "mse")
        self.model = model

In [23]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.001, epsilon=0.1, memory_size=2000, batch_size=32)
train(agent, env, epochs_train, prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 0/21, loss 0.0014, win/lose count 2.5/6.0 (-3.5)
Epoch 1/21, loss 0.0005, win/lose count 1.5/4.0 (-2.5)
Epoch 2/21, loss 0.0089, win/lose count 6.0/5.0 (1.0)
Epoch 3/21, loss 0.0105, win/lose count 12.5/7.0 (5.5)
Epoch 4/21, loss 0.0103, win/lose count 10.5/6.0 (4.5)
Epoch 5/21, loss 0.0038, win/lose count 10.0/2.0 (8.0)
Epoch 6/21, loss 0.0185, win/lose count 11.5/4.0 (7.5)
Epoch 7/21, loss 0.0042, win/lose count 9.5/3.0 (6.5)
Epoch 8/21, loss 0.0066, win/lose count 3.5/3.0 (0.5)
Epoch 9/21, loss 0.0233, win/lose count 13.0/3.0 (10.0)
Epoch 10/21, loss 0.014, win/lose count 9.5/6.0 (3.5)
Epoch 11/21, loss 0.0115, win/lose count 5.5/3.0 (2.5)
Epoch 12/21, loss 0.0129, win/lose count 10.0/5.0 (5.0)
Epoch 13/21, loss 0.0102, win/lose count 13.5/5.0 (8.5)
Epoch 14/21, loss 0.003, win/lose count 9.0/8.0 (1.0)
Epoch 15/21, loss 0.0084, win/lose count 5.5/6.0 (-0.5)
Epoch 16/21, loss 0.0052, win/lose count 9.0/5.0 (4.0)
Epoch 17/21, loss 0.0127, win/lose count 7.0/5.0 (2.0)
Epoch 18/21

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [30]:
### initialize env 
env = Environment(grid_size=size, max_time=T, temperature=0.1)

### parameters
epochs_test = 11

### test CNN
print('Test of the CNN')
agent_cnn = DQN_CNN(size, lr=0.001, epsilon=0.1, memory_size=2000, batch_size=32)
agent_cnn.load(name_weights='cnn_trainmodel.h5', name_model='cnn_trainmodel.json')
test(agent_cnn, env, epochs_test, prefix='cnn_test')

### test FC
print('Test of the FC')
agent_fc = DQN_FC(size, lr=0.001, epsilon=0.1, memory_size=2000, batch_size=32)
agent_fc.load(name_weights='fc_trainmodel.h5', name_model='fc_trainmodel.json')
test(agent_fc,env, epochs_test, prefix='fc_test')

Test of the CNN
Epoch 0, win/lose count 2.0/5.0, average score (-3.0)
Epoch 1, win/lose count 3.0/3.0, average score (-1.5)
Epoch 2, win/lose count 5.5/1.0, average score (0.5)
Epoch 3, win/lose count 5.0/3.0, average score (0.88)
Epoch 4, win/lose count 6.0/1.0, average score (1.7)
Epoch 5, win/lose count 4.5/3.0, average score (1.67)
Epoch 6, win/lose count 8.5/7.0, average score (1.64)
Epoch 7, win/lose count 5.5/3.0, average score (1.75)
Epoch 8, win/lose count 4.0/2.0, average score (1.78)
Epoch 9, win/lose count 6.0/2.0, average score (2.0)
Epoch 10, win/lose count 6.5/4.0, average score (2.05)
Final score: 2.0454545454545454
Test of the FC
Epoch 0, win/lose count 6.5/2.0, average score (4.5)
Epoch 1, win/lose count 3.0/1.0, average score (3.25)
Epoch 2, win/lose count 2.5/3.0, average score (2.0)
Epoch 3, win/lose count 3.0/3.0, average score (1.5)
Epoch 4, win/lose count 5.0/2.0, average score (1.8)
Epoch 5, win/lose count 9.5/5.0, average score (2.25)
Epoch 6, win/lose count 3

In [33]:
HTML(display_videos('cnn_test5.mp4'))

In [32]:
HTML(display_videos('fc_test10.mp4'))

The models tend to not explore the entire board but rather getting stuck. In fact after eating in an area the agents will just hovering around and do nothing. Even if we have a map almost full of cheese (temperature of 0.8 and because we save cheese over poison the map is almsot fully cover of cheese) the models will not explore the rest of the board. Surprisingly when I tested the models with a temperature of 0.1 the models seem to explore a little bit more. But both models achieved some "good" performance since they won some games. I would have expected the CNN model to outperform the FC model but in my case the FC model performed better than the CNN (but the difference between the two is almost nothing when I tested).

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [34]:
def set_epsilon(epsilon, epoch, threshold=0.1):
    temp = epsilon * ((epoch + 1) / (epoch + 2))
    if temp < threshold:
        return threshold
    else:
        return temp

def train_explore(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0
    agent.epsilon = 1.0

    for e in tqdm.tqdm_notebook(range(epoch)):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()

        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        # update the exploration parameter (epsilon)
        agent.epsilon = set_epsilon(agent.epsilon, e) 

        while not game_over:
            # The agent performs an action
            if prefix == 'cnn_train_explore':
                action = agent.learned_act(state.flatten().reshape(1, 5, 5, 3)) # take action with the agent
            else:
                return np.nan
 
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over, prefix)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix + str(e))

        # Update stats
        score += win - lose

        print(f"Epoch {e}/{epoch}, loss {round(np.float64(loss), 4)}, win/lose count {win}/{lose} ({win - lose})")
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        
        self.env = Environment(grid_size=size, max_time=max_time, temperature=temperature)
        self.malus_position = np.zeros((grid_size, grid_size)) # places we already visited
        
    def act(self, action, train=True):
            
        ### first update then env  
        state, reward, game_over = self.env.act(action)
        
        ### get malus if we already visited (x, y), only in train mode
        if train:
            malus_already_visited = - self.malus_position[self.env.x, self.env.y]
        else:
            malus_already_visited = 0
            
        ### update the malus_position array
        self.malus_position[self.env.x, self.env.y] = 0.1

        ### finally we update the reward by considering the malus_position score
        reward += malus_already_visited
    
        
        ### 3-feature state instead of 2
        malus_position_visible = self.malus_position[self.env.x - 2:self.env.x + 3, self.env.y - 2:self.env.y + 3]
        state = np.concatenate((malus_position_visible.reshape(5, 5, 1), state), axis=2)
        
        return state, reward, game_over
    
    def draw(self, e):
        self.env.draw(e)
    
    def reset(self):
        
        ### call the Environment reset function
        state = self.env.reset()
        
        ### reset the malus_position array
        self.malus_position = np.zeros((self.env.grid_size, self.env.grid_size))
        
        ### 3-feature state instead of 2
        malus_position_visible = self.malus_position[self.env.x - 2:self.env.x + 3, self.env.y - 2:self.env.y + 3]
        state = np.concatenate((malus_position_visible.reshape(5, 5, 1), state), axis=2)
        
        return state

In [35]:
# Training
epochs_train = 21
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.001, epsilon=0.1, memory_size=2000, batch_size=32, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore20.mp4'))

Epoch 0/21, loss 0.0105, win/lose count 7.0/26.80000000000011 (-19.80000000000011)
Epoch 1/21, loss 0.0126, win/lose count 16.0/24.800000000000047 (-8.800000000000047)
Epoch 2/21, loss 0.0167, win/lose count 16.5/15.899999999999963 (0.600000000000037)
Epoch 3/21, loss 0.0214, win/lose count 12.5/20.199999999999996 (-7.699999999999996)
Epoch 4/21, loss 0.0099, win/lose count 13.0/21.900000000000045 (-8.900000000000045)
Epoch 5/21, loss 0.0196, win/lose count 19.0/18.800000000000004 (0.19999999999999574)
Epoch 6/21, loss 0.0259, win/lose count 21.5/20.100000000000016 (1.3999999999999844)
Epoch 7/21, loss 0.0092, win/lose count 22.5/18.800000000000015 (3.699999999999985)
Epoch 8/21, loss 0.0237, win/lose count 12.5/22.400000000000034 (-9.900000000000034)
Epoch 9/21, loss 0.0151, win/lose count 21.5/14.899999999999974 (6.600000000000026)
Epoch 10/21, loss 0.0145, win/lose count 16.0/19.099999999999998 (-3.099999999999998)
Epoch 11/21, loss 0.0089, win/lose count 14.5/20.600000000000033 (-6

In [36]:
# Evaluation
test(agent, env, epochs_test, prefix='cnn_test_explore') 
HTML(display_videos('cnn_test_explore9.mp4'))

Epoch 0, win/lose count 23.5/7.0, average score (16.5)
Epoch 1, win/lose count 21.5/7.0, average score (15.5)
Epoch 2, win/lose count 27.0/7.0, average score (17.0)
Epoch 3, win/lose count 15.0/8.0, average score (14.5)
Epoch 4, win/lose count 18.0/5.0, average score (14.2)
Epoch 5, win/lose count 20.0/10.0, average score (13.5)
Epoch 6, win/lose count 29.5/6.0, average score (14.93)
Epoch 7, win/lose count 27.0/7.0, average score (15.56)
Epoch 8, win/lose count 23.0/12.0, average score (15.06)
Epoch 9, win/lose count 8.5/4.0, average score (14.0)
Epoch 10, win/lose count 18.5/9.0, average score (13.59)
Final score: 13.590909090909092


In [37]:
HTML(display_videos('cnn_test_explore1.mp4'))

The model achieve exactly what we wanted

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

It already wins all games. I don't understand "Train a model that mimicks its behavior".

***